## Code

In [1]:
%load_ext ipython_sparql_pandas

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
from random import randint
sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-ClauCy')
sparql.setReturnFormat(JSON)

In [3]:
#from the query results extract a list that contains the counts after splitting on certain attributes
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']

def extractCountAndAttr(result):
    number = []
    listOfObj = []
    listOfPred= []
    #ret = result.queryAndConvert()
    for attr in result["results"]["bindings"]:
        listOfObj.append(attr['o'])
        listOfPred.append(attr['p'])
    return(listOfObj,listOfPred)

def generateQuestion(listOfPred, listOfObj, index, subject):
    flag = True
    if ('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>') in AttrHistory:
        return generateQuestion(listOfPred, listOfObj, index+1, subject)
        
    else:
        AttrHistory.append('<' + str(listOfPred[index]['value']) +'> <'+str(listOfObj[index]['value'])+ '>')
        
    print(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} {listOfObj[index]["value"].split("/")[-1]}?' )
    
    try:
        listOfObj[index]['datatype']
    except:
        flag = False
        
    if flag is True:
        datatype = listOfObj[index]["datatype"].split("#")[-1]
        answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '"'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype)
    else:
        if len(listOfObj[index]["value"].split('/')) == 1:
            answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '"'+str(listOfObj[index]['value'])+ '"')
        else:
            answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '<'+str(listOfObj[index]['value'])+ '>')
    print(answer)
    
    
    if answer:
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'.') 
        else:
            if len(listOfObj[index]["value"].split('/')) == 1:
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+ '".')  
            else:
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>.')    
        
    elif not answer:
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'. }}')
        else:
            if len(listOfObj[index]["value"].split('/')) == 1:
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+ '". }}') 
            else:
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>. }}')   
        
    else:
        return generateQuestion(listOfPred, listOfObj, index+1)
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)

  
def updateQuery(left, PosFilters, NegFilters):
    FilterQuestion =  (f"""
            select (count(*) as ?count) ?p ?o  where 
            {{
            
            {PosFilters}
             ?s ?p ?o .
            {NegFilters}
            }} 
            
            group by ?p ?o 
            ORDER BY ABS( {left} - ?count )
            Limit 25
            
            """) 
        
    return FilterQuestion

In [4]:
def pickSubject():
    #ind=randint(0,10000)
    entities = []
    query = f"""
    select distinct ?s
    where {{
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        entities.append(r['s']['value'])
        #return("<"+r["s"]["value"]+">")
    #data = pd.read_csv('abstracts.csv')
    return entities

def findAnswer(subject,p,o):
    query = f"""
        SELECT ?s ?p ?o
        WHERE {{
        ?s ?p ?o .
            FILTER(?s = {subject})
            FILTER(?p = {p})
            FILTER(?o = {o})
            }}
            """
    #print(subject)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    res = qres["results"]["bindings"]
    return (len(res))>0

In [5]:
def numberleft(PosFilters,NegFilters):
    sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-ClauCy')
    query =  f"""
            select (count(?s) as ?count) where 
            {{
            {PosFilters}            
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
          }}       
            group by ?p ?o 
            ORDER BY DESC(?count )            
            
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        left= (r["count"]["value"])
        return(int(left))

In [6]:
def popentities(PosFilters,NegFilters):
    query =  f"""
            select ?s where 
            {{
            
            {PosFilters}
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
            
            }}
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    return qres["results"]["bindings"]

In [7]:
# resets are variables
def game_reset():
    PosAttr= ['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
    NegAttr= ['']
    AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    left = numberleft("".join(PosAttr), "".join(NegAttr))
    FilterQuestion =  f"""
        select (count(*) as ?count) ?p ?o  where 
        {{

        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
        ?s ?p ?o .
        }} 
        group by ?p ?o 
        ORDER BY ABS( %s - ?count )
        limit 20
        """ %((left/2))
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    return (left, qres, PosAttr, NegAttr, AttrHistory)

In [8]:
subjects_not_found = ['http://yago-knowledge.org/resource/Turkish_language',
 'http://yago-knowledge.org/resource/Chanson',
 'http://yago-knowledge.org/resource/Bristol_City_F.C.',
 'http://yago-knowledge.org/resource/Santa_Cruz,_California',
 'http://yago-knowledge.org/resource/Castle_Rock_Entertainment',
 'http://yago-knowledge.org/resource/Jazz-funk',
 'http://yago-knowledge.org/resource/Metro-Goldwyn-Mayer',
 'http://yago-knowledge.org/resource/Primetime_Emmy_Award_for_Outstanding_Guest_Actor_in_a_Drama_Series',
 'http://yago-knowledge.org/resource/Saturn_Award_for_Best_Costume_Design',
 'http://yago-knowledge.org/resource/Bachelor_of_Arts',
 'http://yago-knowledge.org/resource/Independent_Spirit_Award_for_Best_Director',
 'http://yago-knowledge.org/resource/Stephen_Jackson',
 'http://yago-knowledge.org/resource/Golf',
 'http://yago-knowledge.org/resource/Thai_language',
 'http://yago-knowledge.org/resource/Dennis_Muren',
 'http://yago-knowledge.org/resource/Ames,_Iowa',
 'http://yago-knowledge.org/resource/American_Samoa',
 'http://yago-knowledge.org/resource/Chicago_blues',
 'http://yago-knowledge.org/resource/Redding,_California',
 'http://yago-knowledge.org/resource/Pohang_Steelers',
 'http://yago-knowledge.org/resource/Greenville_County,_South_Carolina',
 'http://yago-knowledge.org/resource/Super_(2010_American_film)',
 'http://yago-knowledge.org/resource/Steve_Jordan_(drummer)',
 'http://yago-knowledge.org/resource/Savannah_Guthrie',
 'http://yago-knowledge.org/resource/Friesland',
 'http://yago-knowledge.org/resource/Poles',
 'http://yago-knowledge.org/resource/East_Java',
 'http://yago-knowledge.org/resource/Primetime_Emmy_Award_for_Outstanding_Lead_Actor_in_a_Comedy_Series',
 'http://yago-knowledge.org/resource/PFC_Botev_Plovdiv',
 'http://yago-knowledge.org/resource/Classical_music',
 'http://yago-knowledge.org/resource/Gary_Sheffield',
 'http://yago-knowledge.org/resource/Ball_State_University',
 'http://yago-knowledge.org/resource/Burnley_F.C.',
 'http://yago-knowledge.org/resource/Ithaca,_New_York',
 'http://yago-knowledge.org/resource/Incheon_United_FC',
 'http://yago-knowledge.org/resource/Preacher',
 'http://yago-knowledge.org/resource/Walt_Disney_Imagineering',
 'http://yago-knowledge.org/resource/Pulitzer_Prize_for_Poetry',
 'http://yago-knowledge.org/resource/Urawa_Red_Diamonds',
 'http://yago-knowledge.org/resource/Baldwin_County,_Alabama',
 'http://yago-knowledge.org/resource/Grammy_Award_for_Best_Dance%2FElectronic_Album',
 'http://yago-knowledge.org/resource/Grammy_Award_for_Best_Pop_Collaboration_with_Vocals',
 'http://yago-knowledge.org/resource/Charlie_Sheen',
 'http://yago-knowledge.org/resource/Touchstone_Pictures',
 'http://yago-knowledge.org/resource/C.D._Veracruz',
 'http://yago-knowledge.org/resource/Yakima,_Washington',
 'http://yago-knowledge.org/resource/GNK_Dinamo_Zagreb',
 'http://yago-knowledge.org/resource/Summit_Entertainment',
 'http://yago-knowledge.org/resource/Underground_hip_hop',
 'http://yago-knowledge.org/resource/Tony_Award_for_Best_Revival_of_a_Musical',
 'http://yago-knowledge.org/resource/Snow_Falling_on_Cedars_(film)',
 'http://yago-knowledge.org/resource/Bethesda,_Maryland',
 'http://yago-knowledge.org/resource/Ferris_Webster',
 'http://yago-knowledge.org/resource/Drenthe',
 'http://yago-knowledge.org/resource/ACF_Fiorentina',
 'http://yago-knowledge.org/resource/Marlene_Dietrich',
 'http://yago-knowledge.org/resource/United_States_Congress',
 'http://yago-knowledge.org/resource/Holy_Roman_Empire',
 'http://yago-knowledge.org/resource/College_Park,_Maryland',
 'http://yago-knowledge.org/resource/Grammy_Award_for_Best_Comedy_Album',
 'http://yago-knowledge.org/resource/Fremantle',
 'http://yago-knowledge.org/resource/The_Road_(2009_film)',
 'http://yago-knowledge.org/resource/General_officer',
 'http://yago-knowledge.org/resource/Stade_Brestois_29',
 'http://yago-knowledge.org/resource/Atlanta',
 'http://yago-knowledge.org/resource/Royal_Medal',
 'http://yago-knowledge.org/resource/Panama_national_football_team',
 'http://yago-knowledge.org/resource/Topaz_(1969_film)',
 'http://yago-knowledge.org/resource/55th_Golden_Globe_Awards',
 'http://yago-knowledge.org/resource/Farnborough_F.C.',
 'http://yago-knowledge.org/resource/Grammy_Award_for_Best_Female_Rock_Vocal_Performance',
 'http://yago-knowledge.org/resource/Celtic_rock',
 'http://yago-knowledge.org/resource/Orduspor',
 'http://yago-knowledge.org/resource/About_Schmidt',
 'http://yago-knowledge.org/resource/Marshall,_Texas',
 'http://yago-knowledge.org/resource/Overijssel',
 'http://yago-knowledge.org/resource/Primetime_Emmy_Award_for_Outstanding_Lead_Actress_in_a_Drama_Series',
 'http://yago-knowledge.org/resource/French_cuisine',
 'http://yago-knowledge.org/resource/Eastleigh_F.C.',
 'http://yago-knowledge.org/resource/Joshua_Jackson',
 'http://yago-knowledge.org/resource/Tours_FC',
 'http://yago-knowledge.org/resource/FC_Urartu',
 'http://yago-knowledge.org/resource/Holy_Roman_Emperor',
 'http://yago-knowledge.org/resource/Boulder,_Colorado',
 'http://yago-knowledge.org/resource/Global_warming',
 'http://yago-knowledge.org/resource/Primetime_Emmy_Award_for_Outstanding_Animated_Program',
 'http://yago-knowledge.org/resource/When_We_Were_Kings',
 'http://yago-knowledge.org/resource/Tony_Award_for_Best_Book_of_a_Musical',
 'http://yago-knowledge.org/resource/Normandy',
 'http://yago-knowledge.org/resource/Charlton_Athletic_F.C.',
 'http://yago-knowledge.org/resource/AC_Horsens',
 'http://yago-knowledge.org/resource/South_Korea_national_under-23_football_team',
 'http://yago-knowledge.org/resource/Lost_in_Translation_(film)',
 'http://yago-knowledge.org/resource/Murfreesboro,_Tennessee',
 'http://yago-knowledge.org/resource/Ukrainians',
 'http://yago-knowledge.org/resource/Lyngby_Boldklub',
 'http://yago-knowledge.org/resource/Boaz_Myhill',
 'http://yago-knowledge.org/resource/Springfield,_Massachusetts',
 'http://yago-knowledge.org/resource/Quincy,_Illinois',
 'http://yago-knowledge.org/resource/Matt_Servitto',
 'http://yago-knowledge.org/resource/Meagan_Good',
 'http://yago-knowledge.org/resource/Jackson_County,_Missouri',
 'http://yago-knowledge.org/resource/British_Raj',
 'http://yago-knowledge.org/resource/Gelderland',
 'http://yago-knowledge.org/resource/A.C._Milan',
 'http://yago-knowledge.org/resource/Alexandria,_Louisiana',
 'http://yago-knowledge.org/resource/Primetime_Emmy_Award_for_Outstanding_Lead_Actress_in_a_Limited_Series_or_Movie',
 'http://yago-knowledge.org/resource/United_States_Conference_of_Catholic_Bishops',
 'http://yago-knowledge.org/resource/Scream_2',
 'http://yago-knowledge.org/resource/SK_Sturm_Graz',
 'http://yago-knowledge.org/resource/Davenport,_Iowa',
 'http://yago-knowledge.org/resource/Bass_(voice_type)',
 'http://yago-knowledge.org/resource/A_Beautiful_Mind_(film)',
 'http://yago-knowledge.org/resource/Hanging',
 'http://yago-knowledge.org/resource/FC_Arsenal_Kyiv',
 'http://yago-knowledge.org/resource/Chinese_Civil_War',
 'http://yago-knowledge.org/resource/Screwball_comedy',
 'http://yago-knowledge.org/resource/Halloween_(2007_film)',
 'http://yago-knowledge.org/resource/OCAD_University',
 'http://yago-knowledge.org/resource/Utrecht_(province)',
 'http://yago-knowledge.org/resource/Kanpur',
 'http://yago-knowledge.org/resource/Greenwood,_Mississippi',
 'http://yago-knowledge.org/resource/A.C._Cesena',
 'http://yago-knowledge.org/resource/United_Artists',
 'http://yago-knowledge.org/resource/Grammy_Award_for_Best_Song_Written_for_Visual_Media',
 'http://yago-knowledge.org/resource/Amy_Smart',
 'http://yago-knowledge.org/resource/The_Church_of_Jesus_Christ_of_Latter-day_Saints',
 'http://yago-knowledge.org/resource/Norwich_University',
 'http://yago-knowledge.org/resource/FC_Chornomorets_Odesa',
 'http://yago-knowledge.org/resource/Emile_Kuri',
 'http://yago-knowledge.org/resource/Brass_instrument',
 'http://yago-knowledge.org/resource/Westminster',
 'http://yago-knowledge.org/resource/Curitiba',
 'http://yago-knowledge.org/resource/Manhattan,_Kansas',
 'http://yago-knowledge.org/resource/Primetime_Emmy_Award_for_Outstanding_Supporting_Actress_in_a_Limited_Series_or_Movie',
 'http://yago-knowledge.org/resource/Warner_Home_Video',
 'http://yago-knowledge.org/resource/Primetime_Emmy_Award_for_Outstanding_Directing_for_a_Comedy_Series',
 'http://yago-knowledge.org/resource/Tagalog_language',
 'http://yago-knowledge.org/resource/Brunswick_County,_North_Carolina',
 'http://yago-knowledge.org/resource/Erotica',
 'http://yago-knowledge.org/resource/38th_Daytime_Emmy_Awards',
 'http://yago-knowledge.org/resource/Red_Tails',
 'http://yago-knowledge.org/resource/Ali_Larter',
 'http://yago-knowledge.org/resource/Primetime_Emmy_Award_for_Outstanding_Supporting_Actress_in_a_Comedy_Series',
 'http://yago-knowledge.org/resource/AGOVV_Apeldoorn',
 'http://yago-knowledge.org/resource/Kate_Greenaway_Medal',
 'http://yago-knowledge.org/resource/Bicentennial_Man_(film)',
 'http://yago-knowledge.org/resource/25th_United_States_Congress',
 'http://yago-knowledge.org/resource/Bloomington,_Illinois',
 'http://yago-knowledge.org/resource/Grammy_Award_for_Best_Instrumental_Soloist(s)_Performance_(with_orchestra)',
 "http://yago-knowledge.org/resource/It's_a_Mad,_Mad,_Mad,_Mad_World",
 'http://yago-knowledge.org/resource/15th_Satellite_Awards',
 'http://yago-knowledge.org/resource/CBS_Schoolbreak_Special',
 'http://yago-knowledge.org/resource/London_Borough_of_Newham',
 'http://yago-knowledge.org/resource/Funk_metal',
 'http://yago-knowledge.org/resource/Confederate_States_of_America',
 'http://yago-knowledge.org/resource/Crystal_Palace_F.C.',
 'http://yago-knowledge.org/resource/Swansea',
 'http://yago-knowledge.org/resource/Adelaide_United_FC',
 'http://yago-knowledge.org/resource/Jenna_Ushkowitz',
 'http://yago-knowledge.org/resource/28th_United_States_Congress',
 'http://yago-knowledge.org/resource/Lawrence,_Massachusetts',
 'http://yago-knowledge.org/resource/FC_Volga_Nizhny_Novgorod',
 'http://yago-knowledge.org/resource/Chicken',
 'http://yago-knowledge.org/resource/FC_St._Gallen',
 'http://yago-knowledge.org/resource/Metro_Manila',
 'http://yago-knowledge.org/resource/Puerto_Rico_national_football_team',
 'http://yago-knowledge.org/resource/Grammy_Award_for_Best_Rock_Song',
 'http://yago-knowledge.org/resource/Chief_executive_officer',
 'http://yago-knowledge.org/resource/Teri_Polo',
 'http://yago-knowledge.org/resource/Madison,_Wisconsin',
 'http://yago-knowledge.org/resource/Damian_Marley',
 'http://yago-knowledge.org/resource/Bromley',
 'http://yago-knowledge.org/resource/Parkersburg,_West_Virginia',
 'http://yago-knowledge.org/resource/Greece_national_football_team',
 'http://yago-knowledge.org/resource/Royal_Oak,_Michigan',
 'http://yago-knowledge.org/resource/Primetime_Emmy_Award_for_Outstanding_Lead_Actor_in_a_Limited_Series_or_Movie',
 'http://yago-knowledge.org/resource/Racism',
 'http://yago-knowledge.org/resource/Seville',
 'http://yago-knowledge.org/resource/Colo-Colo',
 'http://yago-knowledge.org/resource/Warren,_Ohio',
 'http://yago-knowledge.org/resource/Persepolis_(film)',
 'http://yago-knowledge.org/resource/Kensington',
 'http://yago-knowledge.org/resource/James_Marsden',
 'http://yago-knowledge.org/resource/Requiem_for_a_Dream',
 'http://yago-knowledge.org/resource/Singer-songwriter',
 'http://yago-knowledge.org/resource/Yunjin_Kim',
 'http://yago-knowledge.org/resource/Eastern_Michigan_University',
 'http://yago-knowledge.org/resource/Visalia,_California',
 'http://yago-knowledge.org/resource/Council_Bluffs,_Iowa',
 'http://yago-knowledge.org/resource/Latin_jazz',
 'http://yago-knowledge.org/resource/Star_Alliance',
 'http://yago-knowledge.org/resource/Great_Falls,_Montana',
 'http://yago-knowledge.org/resource/Zambia_national_football_team',
 'http://yago-knowledge.org/resource/Derby_County_F.C.',
 'http://yago-knowledge.org/resource/Stirling_Albion_F.C.',
 'http://yago-knowledge.org/resource/Robots_(2005_film)',
 'http://yago-knowledge.org/resource/Christopher_Young',
 'http://yago-knowledge.org/resource/F.C._Copenhagen',
 'http://yago-knowledge.org/resource/Selangor']

no_ab = ['AGOVV_Apeldoorn',
'Global_warming',
'Grammy_Award_for_Best_Dance%2FElectronic_Album',
'PFC_Botev_Plovdiv',
'Primetime_Emmy_Award_for_Outstanding_Lead_Actor_in_a_Limited_Series_or_Movie',
'Primetime_Emmy_Award_for_Outstanding_Lead_Actress_in_a_Limited_Series_or_Movie',
'Primetime_Emmy_Award_for_Outstanding_Supporting_Actress_in_a_Limited_Series_or_Movie',
'Warner_Home_Video']
subs = []
for i in subjects_not_found:
    if i.split('/')[-1] in no_ab:
        print(i)
        continue 
    else:
        subs.append(i)

http://yago-knowledge.org/resource/PFC_Botev_Plovdiv
http://yago-knowledge.org/resource/Grammy_Award_for_Best_Dance%2FElectronic_Album
http://yago-knowledge.org/resource/Global_warming
http://yago-knowledge.org/resource/Primetime_Emmy_Award_for_Outstanding_Lead_Actress_in_a_Limited_Series_or_Movie
http://yago-knowledge.org/resource/Primetime_Emmy_Award_for_Outstanding_Supporting_Actress_in_a_Limited_Series_or_Movie
http://yago-knowledge.org/resource/Warner_Home_Video
http://yago-knowledge.org/resource/AGOVV_Apeldoorn
http://yago-knowledge.org/resource/Primetime_Emmy_Award_for_Outstanding_Lead_Actor_in_a_Limited_Series_or_Movie


In [ ]:
import random
import pandas as pd
random.seed(10)
subjects = subs
wincount = 0
subjects_lost = []
gamesplayed = 0
lostbybreak = 0
breakskip = 0
totalgames = len(subjects)

while gamesplayed < totalgames:
    # Resets all variables
    try:
        left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
        i=0
        outofguess = []
        print(gamesplayed)
        # pick subject
        subject = "<" + subjects[gamesplayed] + ">"
        print(subject)

        while i < 20:

            # list of Pred and Obj for questions
            listOfObj ,listOfPred = extractCountAndAttr(qres)

            # if only 1 popular entity is left break
            if left<=1:
                break

            # ask question and add filters to query
            print('Question: %s' %(i+1))
            PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, 0, subject)

            # number of popular entities left
            left = numberleft(PosFilters, NegFilters)
            print('left -',left)

            # run query
            sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-ClauCy')
            query = updateQuery(left/2,PosFilters, NegFilters)
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            qres = sparql.queryAndConvert()

            i+=1

        # Prints answer
        if i != 20:
            for attr in popentities(PosFilters, NegFilters):
                for x in [attr['s']['value']][0:10]:
                    answer = (x.split("/")[-1])
            print("Your Answer is: " + answer)
            if str(answer.split("/")[-1] + ">") == str(subject.split("/")[-1]):
                wincount += 1
                gamesplayed += 1
                print("Win!")
                print('%s/%s' %(wincount,gamesplayed))
                print()
            else:
                gamesplayed += 1
                subjects_lost.append(subject)
                print(gamesplayed)
                print(subject)
                print("Lose...")
                print('%s/%s' %(wincount,gamesplayed))
                print()

        # ran out of questions
        else:
            #print('Final Question:')
            for j in popentities(PosFilters, NegFilters):
                    outofguess.append(f'{j["s"]["value"].split("/")[-1]}')
            guess = random.choice(outofguess)
            print("Is it " + guess + "?")
            if str(subject.split("/")[-1]) == str(guess + ">"):
                wincount += 1
                gamesplayed += 1
                print('Win!')
                print('%s/%s' %(wincount,gamesplayed))
                print()
            else:
                gamesplayed += 1
                subjects_lost.append(subject)
                print(gamesplayed)
                print(subject)
                print('Lose...')
                print('%s/%s' %(wincount,gamesplayed))
                print()
        breakskip = 0
    except:
        continue
            
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")

0
<http://yago-knowledge.org/resource/Turkish_language>
Question: 1
Does the thing you are looking for have the attribute: 22-rdf-syntax-ns#type Person?
False
left - 8409
Question: 2
Does the thing you are looking for have the attribute: 22-rdf-syntax-ns#type Organization?
False
left - 4918
Question: 3
Does the thing you are looking for have the attribute: 22-rdf-syntax-ns#type CreativeWork?
False
left - 2429
Question: 4
Does the thing you are looking for have the attribute: 22-rdf-syntax-ns#type Place?
False
left - 1317
Question: 5
Does the thing you are looking for have the attribute: 22-rdf-syntax-ns#type Event?
False
left - 1128
Question: 6
Does the thing you are looking for have the attribute: 22-rdf-syntax-ns#type Occupation?
False
left - 958
Question: 7
Does the thing you are looking for have the attribute: 22-rdf-syntax-ns#type MedicalCondition?
False
left - 879
Question: 8
Does the thing you are looking for have the attribute: 22-rdf-syntax-ns#type award?
False
left - 802
Ques

In [ ]:
#9/21 - ClauCy
#11/21 - Stanford

In [ ]:
sub = []
for i in subjects_lost:
    i = i.split('<')[1]
    i = i.split('>')[0]
    sub.append("%s" %(i))
sub

In [ ]:
import random
import pandas as pd
random.seed(10)
#subjects = pickSubject()
wincount = 0
subjects_lost = []
gamesplayed = 0
lostbybreak = 0
breakskip = 0
#totalgames = len(subjects)

left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
i=0
outofguess = []
subject = "<http://yago-knowledge.org/resource/Jesse_Metcalfe>"
print(subject)

while i < 20:

    # list of Pred and Obj for questions
    listOfObj ,listOfPred = extractCountAndAttr(qres)

    # if only 1 popular entity is left break
    if left<=1:
        break

    # ask question and add filters to query
    #print('Question: %s' %(i+1))
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, 0, subject)

    # number of popular entities left
    left = numberleft(PosFilters, NegFilters)

    # run query
    sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-ClauCy')
    query = updateQuery(left/2,PosFilters, NegFilters)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.queryAndConvert()

    i+=1

# Prints answer
if i != 20:
    for attr in popentities(PosFilters, NegFilters):
        for x in [attr['s']['value']][0:10]:
            answer = (x.split("/")[-1])
    print("Your Answer is: " + answer)
    if str(answer.split("/")[-1] + ">") == str(subject.split("/")[-1]):
        wincount += 1
        gamesplayed += 1
        #print("Win!")
        #print('%s/%s' %(wincount,gamesplayed))
        #print()
    else:
        gamesplayed += 1
        subjects_lost.append(subject)
        print("Lose...")
        print('%s/%s' %(wincount,gamesplayed))
        print()

# ran out of questions
else:
    print('Final Question:')
    for j in popentities(PosFilters, NegFilters):
            outofguess.append(f'{j["s"]["value"].split("/")[-1]}')
    guess = random.choice(outofguess)
    print("Is it " + guess + "?")
    if str(subject.split("/")[-1]) == str(guess + ">"):
        wincount += 1
        gamesplayed += 1
        #print('Win!')
        #print('%s/%s' %(wincount,gamesplayed))
        #print()
    else:
        gamesplayed += 1
        subjects_lost.append(subject)
        print('Lose...')
        print('%s/%s' %(wincount,gamesplayed))
        print()
breakskip = 0
   
            
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")

#### Seeds used:
- 10
- ??
- ??

#### Time started:
- 10
    - base model
        - 22:50


In [ ]:
len('ALL'.split('/'))